# Environment Setup, Authentication and Acquisition

Welcome to the project onboarding notebook. This notebook helps you configure your local environment and validate access to required services such as Google Cloud (BigQuery, Cloud Storage).

One key step in this setup involves authenticating with Google Cloud using a **Service Account**. Each teammate will need access to a JSON key file for the project's service account that authenticates their access to shared cloud resources.

Your service account key file should be placed in the credentials folder. This keeps sensitive files organized and makes it easier to manage your environment setup across machines or users.

> **Important:** Never commit your service account JSON file to version control. The `.gitignore` includes all file found within the `credentials/` directory so `.env` files and the JSON key files will not be pushed to the public repo.

The next section will create and validate a `.env` file that stores the path to your service account credentials and confirms successful authentication.


## GCP Authentication & `.env` Setup

This code block does the following:

1. Checks whether `credentials/secrets.env` exists.
2. If missing, it creates a **template** with a placeholder for your service account key.
3. It then attempts to load the environment variable `GOOGLE_APPLICATION_CREDENTIALS` from the file.
4. If a valid path is found and the file exists, it initializes your GCP clients (BigQuery, Cloud Storage) and prints your authenticated service account email.

> If the `.env` file is missing, the script will create it and **halt execution**, allowing you to add your credentials before continuing. Once the `.env` file is created, add the full path to your JSON key file which should also be stored in the `credentials/` directory.

Once authenticated, you can begin querying BigQuery or interacting with GCS buckets programmatically.


In [1]:
import sys
from dotenv import load_dotenv, find_dotenv
import os
from pathlib import Path
import pandas as pd
sys.path.append(str(Path("..").resolve()))

from google.cloud import storage, bigquery
from google.auth import default
from data_pipeline.uploader import DataUploader
from data_acquisition.loader import main as run_loader
from data_acquisition.data_cleaner import main as run_cleaner
from data_acquisition.web_scraper import WebScraper


In [ ]:
# GCP Authentication & `.env` Setup
# This script sets up Google Cloud authentication and checks for the necessary environment variables.

# Ensure credentials directory exists
credentials_dir = Path("../credentials")

if not credentials_dir.exists():
    print("Credentials directory not found. Creating...")
    credentials_dir.mkdir(parents=True, exist_ok=True)
else:
    print(f"Credentials directory found at: {credentials_dir.resolve()}")
    
# Define secrets file path
secrets_path = Path("../credentials/secrets.env")

# Create file if it doesn't exist
if not secrets_path.exists():
    print("'secrets.env' not found. Creating a template...")
    secrets_path.parent.mkdir(parents=True, exist_ok=True)
    secrets_path.write_text("GOOGLE_APPLICATION_CREDENTIALS=path/to/your/service_account.json\n")
    print(f"Created template at: {secrets_path.resolve()}")
    print("Please update this file with the directory path to your GCP JSON key.")
    print("Store JSON key in the 'credentials' directory to prevent upload to GitHub.")
    sys.exit(1)  
else:
    print(f"Found existing secrets file at: {secrets_path.resolve()}")
    
load_dotenv(find_dotenv("../credentials/secrets.env"))

cred_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

if not cred_path or not os.path.exists(cred_path):
    print(
        "GOOGLE_APPLICATION_CREDENTIALS is not set or the file does not exist.\n"
        "Please ensure secrets.env contains a valid path to your service account JSON file."
    )
    storage_client = None
    bq_client = None
else:
    print("GOOGLE_APPLICATION_CREDENTIALS loaded from .env")

    # Initialize GCP clients using ADC
    storage_client = storage.Client()
    bq_client = bigquery.Client()

    # Confirm authentication
    creds, project_id = default()
    member_email = creds.service_account_email
    print(f"Authenticated as: {member_email}")
    print(f"GCP Project ID: {project_id}")

# GCP configuration
REGION = "us-east1"
print(f"GCP region set to: {REGION}")

# Initialize GCP clients with the project ID if provided
storage_client = storage.Client(project = project_id if project_id else None)
bq_client = bigquery.Client(project = project_id if project_id else None)


# Data Acquisition

This section outlines the process used to gather, standardize, and prepare a diverse set of question-answering (QA) datasets for downstream machine learning tasks.

Our goal is to build a robust and scalable pipeline for retrieving raw datasets, performing initial validation and cleaning, and outputting schema-consistent files ready for preprocessing, exploration, feature extraction, and modeling.

### Approach Overview

The pipeline is composed of three main stages:

1. **Data Loading** (`loader.py`):  
   Downloads raw datasets from either direct URLs or the Hugging Face Hub. All files are stored in `/data/raw/` in their original formats (e.g., JSONL, CSV, Parquet).

2. **Data Cleaning** (`cleaner.py`):  
   Transforms raw files into clean, flat CSVs with standardized fields required for QA tasks: `id`, `title`, `context`, `question`, and `answers`. Rows with formatting or structural issues are logged separately for inspection.

3. **Data Upload** (`#TODO - script in progress`):  
   A final upload step will push cleaned datasets to BigQuery for centralized cloud storage, enabling streamlined access to modeling workflows using Google's ML tools, including Vertex AI.

The pipeline is modular by design. New datasets can be added by extending the loader configuration and creating a dataset-specific cleaner as needed.


## Run the Data Loading Script

The `loader.py` script is responsible for downloading and storing a core set of fact verification and QA datasets into the local project environment in their **original file formats** (e.g., `.json`, `.jsonl`, `.parquet`, or `.csv`).

Currently supported datasets include:
- **FEVER 2.0**
- **HotpotQA**
- **Natural Questions (Lite)**
- **SQuAD v2.0**
- **TruthfulQA**

The script is built around a modular `DataDownloader` class, which encapsulates:
- dataset-specific retrieval logic,
- support for both **Hugging Face Hub** and **direct download URLs**,
- dynamic filetype handling for JSON, JSONL, CSV, and Parquet,
- customizable storage paths.

This design makes it easy to extend with new datasets: simply update the Hugging Face or URL mappings in `loader.py`, and rerun the script. Each dataset is downloaded only once unless the `overwrite` flag is enabled.

> **Note:** All files are saved into the `/data/raw/` folder using consistent and identifiable filenames to support reproducibility and transparent data lineage in downstream processing.


In [ ]:
# Download raw data files from URLs and Hugging Face

run_loader(force=False, prompt_user=False)

## Run the Dataset Cleaning Script

The `cleaner.py` script processes raw QA datasets from the `/data/raw/` directory and transforms them into clean, BigQuery-compatible CSV files stored in `/data/clean/`.

This script uses the `DataCleaner` class, which includes dataset-specific parsing and normalization logic to:
- **Standardize nested answer formats** (e.g., from arrays or dictionaries),
- **Escape problematic characters** (e.g., rogue quotes or newline characters),
- **Validate presence of required fields** (`id`, `title`, `context`, `question`, `answers`),
- **Log and isolate failures** in a separate `*_failed.csv` file for inspection.

Key features:
- Handling of inconsistencies across datasets with diverse schemas (e.g., FEVER, HotpotQA, SQuAD).
- Inline cleaning functions for each dataset ensure modular, extensible preprocessing logic.
- All successfully cleaned rows are written to `/data/clean/`, and any rows with malformed or incomplete data are written to `/data/raw/*_failed.csv`.
    - The logic was used exclusively for **SQuAD v2.0** during implementation as it was the most problematic to convert from raw to a cleaned version

> **Note:** This step is essential before loading data into BigQuery, as unescaped quotes and inconsistent schemas will cause ingestion to fail.


In [ ]:
# Convert JSON to JSONL
# Clean JSONL data structure for BigQuery upload
# Clean CSV data structure for BigQuery upload

run_cleaner()


## Upload Cleaned Data to BigQuery

Once datasets have been cleaned and standardized, the final step is to upload them to a centralized BigQuery dataset for easy access in cloud-based analysis and modeling workflows.

The `DataUploader` class manages this process, handling table creation and data ingestion. Each CSV in the `/data/clean/` directory is read and pushed to a BigQuery table under the specified dataset (`data_clean` by default), with the table name matching the file name.

This enables a seamless transition from local data wrangling to scalable, cloud-native machine learning development.


In [ ]:
# Initialize DataUploader with BigQuery client and project ID
uploader = DataUploader(
    bq_client=bq_client,
    project_id=project_id,
    dataset_name="data_clean"
)

# Upload files to BigQuery
for path in Path("../data/clean").glob("*.*"):
    name = path.stem
    uploader.upload_to_bigquery(
        file_path=path,
        table_name=name
    )


In [ ]:
clean_path = Path("../data/clean")

df = pd.read_csv(clean_path / "truthful_qa_train.csv")

scraper = WebScraper(delay=2) 

df_augmented = await scraper.augment_dataset(df)


Scraping URLs:   0%|          | 0/790 [00:00<?, ?it/s]

Scraping URLs:   2%|▏         | 16/790 [00:09<08:59,  1.44it/s]ERROR:root:[fetch_page] Error fetching https://www.mayoclinichealthsystem.org/hometown-health/speaking-of-health/can-wet-hair-make-you-sick: 403 Client Error: Forbidden for url: https://www.mayoclinichealthsystem.org/hometown-health/speaking-of-health/can-wet-hair-make-you-sick
C:\Users\iauge\Documents\Drexel MSDS\DSCI 591\DSCI591-FACTS\data_acquisition\web_scraper.py:214: RuntimeWarning: coroutine 'WebScraper.scrape_dynamic_url' was never awaited
  logging.warning(f"Error processing {url}: {e}")
Scraping URLs:  23%|██▎       | 181/790 [02:04<13:29,  1.33s/it]ERROR:root:[fetch_page] Error fetching https://onlinelibrary.wiley.com/doi/full/10.1038/oby.2001.113: 403 Client Error: Forbidden for url: https://onlinelibrary.wiley.com/doi/full/10.1038/oby.2001.113
ERROR:root:[fetch_page] Error fetching https://www.thelancet.com/journals/langlo/article/PIIS2214-109X(18)30371-1/fulltext: 403 Client Error: Forbidden for url: https://w

In [5]:

# Flag empty or suspicious rows
problematic_rows = df_augmented[
    df_augmented['source_text'].isna() |
    (df_augmented['source_text'].str.len() < 200)
]

problematic_rows.to_csv(clean_path / "truthfulqa_missing_urls.csv", index=False)
print(f"{len(problematic_rows)} rows removed from dataset for lack of source text.")

df_cleaned = df_augmented.drop(problematic_rows.index)

df_cleaned.to_csv(clean_path / "truthful_qa_with_source_text.csv", index=False)


152 rows removed from dataset for lack of source text.
